In [18]:
############################## homograpy & 위험영역 지정 ###########################
import cv2 as cv
import numpy as np
from PIL import Image
import math
from shapely.geometry import Point, Polygon # 기하학적 물체 표현

result = [0,0,0,0,0,0,0,0]

### 차도영역 지정 -> 변환행렬 구하기

#Read source image
main_im = cv.imread('test_1.jpg')

#Four corners of the book in source image
road_x = np.array([[12, 1132, 1], [12, 2148, 1], [3836, 2156, 1], [3836, 888, 1], [2640, 736, 1], [2340, 520, 1], [2552, 508, 1], [2488, 428, 1],
              [2200, 424, 1], [2036, 276, 1], [1892, 268, 1], [1896, 436, 1], [1596, 440, 1], [1588, 508, 1], [1896, 520, 1], [1908, 720, 1], [1428, 944, 1]])

#Read destination image
main_im_ = cv.imread('naver_map.png')

#Four corners of the book in destination image
road_x_ = np.array([[2912, 1464, 1], [3352, 1272, 1], [3380, 980, 1], [2792, 664, 1], [2504, 944, 1], [1956, 964, 1], [1752, 816, 1], [1388, 812, 1],
               [1224, 952, 1], [956, 960, 1], [956, 1096, 1], [1304, 1132, 1], [1384, 1280, 1], [1664, 1244, 1], [1816, 1148, 1], [2496, 1100, 1], [2768, 1212, 1]])

#Calculate Homography
h_,status = cv.findHomography(road_x,road_x_)

print("변환행렬 h_ :\n", h_)

### 위험영역 지정

ROI_1 = Polygon([[12, 1132, 1], [12, 2148, 1], [3836, 2156, 1], [3836, 888, 1]])
ROI_2 = Polygon([[12, 1132, 1], [12, 2148, 1], [3836, 2156, 1], [3836, 888, 1]])
ROI_3 = Polygon([[12, 1132, 1], [12, 2148, 1], [3836, 2156, 1], [3836, 888, 1]])
####################################################################################

with open(predict_result_path, 'w') as f:
    in_ped = [] # 차도 영역 내 보행자 리스트
    case = 0
    for bbox in bboxes_pr:
        coor = np.array(bbox[:4], dtype=np.int32)
        score = bbox[4]
        class_ind = int(bbox[5])
        class_name = self.classes[class_ind]
        score = '%.4f' % score
        xmin, ymin, xmax, ymax = list(map(str, coor))
        bbox_mess = ' '.join([class_name, score, xmin, ymin, xmax, ymax]) + '\n'
        #bbox_mess : vehicle 0.7585 67 280 582 618
        
        ################## 여기에 들어감 ####################
        point = []
        
        # vehicle -> 박스 중점 / 나머지 -> 하단 중점
        if class_name == 'vehicle':
            point.append([int((xmin+xmax)/2), int((ymin+ymax)/2)])

            new_veh_x = (h_[0][0]*point[0] + h_[0][1]*point[1] + h_[0][2]) / (h_[2][0]*point[0] + h_[2][1]*point[1] + 1)
            new_veh_y = (h_[1][0]*point[0] + h_[1][1]*point[1] + h_[1][2]) / (h_[2][0]*point[0] + h_[2][1]*point[1] + 1)
            
            for j in range(len(in_ped)):
                dis = math.sqrt((in_ped[j][0] - new_veh_x)**2 + ((in_ped[j][1] - new_veh_y)**2))
                if dis < 200:
                    case = 1
                    break
                else:
                    case = 0

            if case == 0:
                print("안전! 충돌 위험한 보행자가 없습니다. (위치 좌표 :", new_veh_x, ",", new_veh_y, ")")
            else:
                print("위험! 충돌 위험한 보행자가 있습니다! (위치 좌표 :", new_veh_x, ",", new_veh_y, ")")

        elif class_name == 'person':
            point.extend([int((xmin+xmax)/2), int(ymax)])

            ### 차도 영역 내 보행자 판단
            ROI_co = road_x # 차도 꼭지점
            ped = Point(point[0], point[1])
            ROI = Polygon(ROI_co)

            if ped.within(ROI) == 1:
                print("경고! 차도 내 보행자가 위치합니다!")
                new_ped_x = (h_[0][0]*point[0] + h_[0][1]*point[1] + h_[0][2]) / (h_[2][0]*point[0] + h_[2][1]*point[1] + 1)
                new_ped_y = (h_[1][0]*point[0] + h_[1][1]*point[1] + h_[1][2]) / (h_[2][0]*point[0] + h_[2][1]*point[1] + 1)
                in_ped.append([int(new_ped_x),int(new_ped_y)])
            else:
                print("안전! 차도 내 보행자가 없습니다.")

            ### 위험 지역 내 보행자 판단
            if ped.within(ROI_1) or ped.within(ROI_2) or ped.within(ROI_3) == 1:
                print("경고! 위험구역 내 보행자가 위치합니다!")
            else:
                print("안전! 위험구역 내 보행자가 없습니다.")


        elif class_name == 'cycle':
            point.extend([int((xmin+xmax)/2), int(ymax)])

            ### 차도 영역 내 자전거 판단
            ROI_co = road_x # 차도 꼭지점
            cyc = Point(point[0], point[1])
            ROI = Polygon(ROI_co)

            if cyc.within(ROI) == 1:
                print("경고! 차도 내 자전거가 위치합니다!")
            else:
                print("안전! 차도 내 자전거가 없습니다.")

            ### 위험 지역 내 자전거 판단
            if cyc.within(ROI_1) or cyc.within(ROI_2) or cyc.within(ROI_3) == 1:
                print("경고! 위험구역 내 자전거가 위치합니다!")
            else:
                print("안전! 위험구역 내 자전거가 없습니다.")


        elif class_name == 'kickboard':
            point.extend([int((xmin+xmax)/2), int(ymax)])

            ### 차도 영역 내 킥보드 판단
            ROI_co = road_x # 차도 꼭지점
            kick = Point(point[0], point[1])
            ROI = Polygon(ROI_co)

            if kick.within(ROI) == 1:
                print("경고! 차도 내 킥보드가 위치합니다!")
            else:
                print("안전! 차도 내 킥보드가 없습니다.")

            ### 위험 지역 내 킥보드 판단
            if kick.within(ROI_1) or kick.within(ROI_2) or kick.within(ROI_3) == 1:
                print("경고! 위험구역 내 킥보드가 위치합니다!")
            else:
                print("안전! 위험구역 내 킥보드가 없습니다.")


        elif class_name == 'motorbike':
            point.extend([int((xmin+xmax)/2), int(ymax)])

            ROI_co = road_x # 차도 꼭지점
            moto = Point(point[0], point[1])
            ROI = Polygon(ROI_co)

            ### 위험 지역 내 오토바이 판단
            if moto.within(ROI_1) or moto.within(ROI_2) or moto.within(ROI_3) == 1:
                print("경고! 위험구역 내 오토바이가 위치합니다!")
            else:
                print("안전! 위험구역 내 오토바이가 없습니다.")
        
        #####################################################
        
        f.write(bbox_mess)
        print('\t' + str(bbox_mess).strip())

NameError: name 'predict_result_path' is not defined

In [10]:
import cv2 as cv
import numpy as np
from PIL import Image
import math
from shapely.geometry import Point, Polygon # 기하학적 물체 표현

result = [0,0,0,0,0,0,0,0]

### 차도영역 지정 -> 변환행렬 구하기

#Read source image
main_im = cv.imread('test_1.jpg')

#Four corners of the book in source image
road_x = np.array([[12, 1132, 1], [12, 2148, 1], [3836, 2156, 1], [3836, 888, 1],
                   [2640, 736, 1], [2340, 520, 1], [2552, 508, 1], [2488, 428, 1],
                   [2200, 424, 1], [2036, 276, 1], [1892, 268, 1], [1896, 436, 1],
                   [1596, 440, 1], [1588, 508, 1], [1896, 520, 1], [1908, 720, 1],
                   [1428, 944, 1]])

#Read destination image
main_im_ = cv.imread('naver_map.png')

#Four corners of the book in destination image
road_x_ = np.array([[2912, 1464, 1], [3352, 1272, 1], [3380, 980, 1], [2792, 664, 1],
                    [2504, 944, 1], [1956, 964, 1], [1752, 816, 1], [1388, 812, 1],
                    [1224, 952, 1], [956, 960, 1], [956, 1096, 1], [1304, 1132, 1],
                    [1384, 1280, 1], [1664, 1244, 1], [1816, 1148, 1], [2496, 1100, 1],
                    [2768, 1212, 1]])

#Calculate Homography
h_,status = cv.findHomography(road_x,road_x_)

print("변환행렬 h_ :\n", h_)

### 위험영역 지정

ROI_1 = Polygon([[12, 1132, 1], [12, 2148, 1], [3836, 2156, 1], [3836, 888, 1]])
ROI_2 = Polygon([[12, 1132, 1], [12, 2148, 1], [3836, 2156, 1], [3836, 888, 1]])
ROI_3 = Polygon([[12, 1132, 1], [12, 2148, 1], [3836, 2156, 1], [3836, 888, 1]])

# Ex)
class_name = 'vehicle'
xmin = 1900
ymin = 900
xmax = 2100
ymax = 1100

in_ped = [[100,100],[200,200],[300,300]] # 차도 영역 내 보행자 리스트
veh_pt = [[400,400],[600,600],[700,700],[800,800]] # 차량 중점 좌표 list

print(h_[0][0])
print(point[0][0])


point = [] # 확인할 좌표

# vehicle -> 박스 중점 / 나머지 -> 하단 중점
if class_name == 'vehicle':
    point.append([int((xmin+xmax)/2), int((ymin+ymax)/2)])
    
    new_veh_x = (h_[0][0]*point[0] + h_[0][1]*point[1] + h_[0][2]) / (h_[2][0]*point[0] + h_[2][1]*point[1] + 1)
    new_veh_y = (h_[1][0]*point[0] + h_[1][1]*point[1] + h_[1][2]) / (h_[2][0]*point[0] + h_[2][1]*point[1] + 1)
    veh_pt.append([int(new_veh_x),int(new_veh_y)])
    
elif class_name == 'pedestrian':
    point.extend([int((xmin+xmax)/2), int(ymax)])
    
    ### 차도 영역 내 보행자 판단
    ROI_co = road_x # 차도 꼭지점
    ped = Point(point[0], point[1])
    ROI = Polygon(ROI_co)

    if ped.within(ROI) == 1:
        print("경고! 차도 내 보행자가 위치합니다!")
        new_ped_x = (h_[0][0]*point[0] + h_[0][1]*point[1] + h_[0][2]) / (h_[2][0]*point[0] + h_[2][1]*point[1] + 1)
        new_ped_y = (h_[1][0]*point[0] + h_[1][1]*point[1] + h_[1][2]) / (h_[2][0]*point[0] + h_[2][1]*point[1] + 1)
        in_ped.append([int(new_ped_x),int(new_ped_y)])
    else:
        print("안전! 차도 내 보행자가 없습니다.")
    
    ### 위험 지역 내 보행자 판단
    if ped.within(ROI_1) or ped.within(ROI_2) or ped.within(ROI_3) == 1:
        print("경고! 위험구역 내 보행자가 위치합니다!")
    else:
        print("안전! 위험구역 내 보행자가 없습니다.")
    
    
elif class_name == 'cycle':
    point.extend([int((xmin+xmax)/2), int(ymax)])
    
    ### 차도 영역 내 자전거 판단
    ROI_co = road_x # 차도 꼭지점
    cyc = Point(point[0], point[1])
    ROI = Polygon(ROI_co)

    if cyc.within(ROI) == 1:
        print("경고! 차도 내 자전거가 위치합니다!")
    else:
        print("안전! 차도 내 자전거가 없습니다.")
        
    ### 위험 지역 내 자전거 판단
    if cyc.within(ROI_1) or cyc.within(ROI_2) or cyc.within(ROI_3) == 1:
        print("경고! 위험구역 내 자전거가 위치합니다!")
    else:
        print("안전! 위험구역 내 자전거가 없습니다.")
    
    
elif class_name == 'kickboard':
    point.extend([int((xmin+xmax)/2), int(ymax)])
    
    ### 차도 영역 내 킥보드 판단
    ROI_co = road_x # 차도 꼭지점
    kick = Point(point[0], point[1])
    ROI = Polygon(ROI_co)

    if kick.within(ROI) == 1:
        print("경고! 차도 내 킥보드가 위치합니다!")
    else:
        print("안전! 차도 내 킥보드가 없습니다.")
    
    ### 위험 지역 내 킥보드 판단
    if kick.within(ROI_1) or kick.within(ROI_2) or kick.within(ROI_3) == 1:
        print("경고! 위험구역 내 킥보드가 위치합니다!")
    else:
        print("안전! 위험구역 내 킥보드가 없습니다.")
    
    
elif class_name == 'motocycle':
    point.extend([int((xmin+xmax)/2), int(ymax)])
    
    ROI_co = road_x # 차도 꼭지점
    moto = Point(point[0], point[1])
    ROI = Polygon(ROI_co)

    ### 위험 지역 내 오토바이 판단
    if moto.within(ROI_1) or moto.within(ROI_2) or moto.within(ROI_3) == 1:
        print("경고! 위험구역 내 오토바이가 위치합니다!")
    else:
        print("안전! 위험구역 내 오토바이가 없습니다.")

### 충돌 알림(차도영역 한정) -> 사람 우선 list화 - 사람을 기준으로 차량과의 거리 반복 계산
for i in range(len(veh_pt)):
    for j in range(len(in_ped)):
        dis = math.sqrt((in_ped[j][0] - veh_pt[i][0])**2 + ((in_ped[j][1] - veh_pt[i][1])**2))
        if dis < 200:
            case = 1
            break
        else:
            case = 0
    if case == 1: break

if case == 0:
    print("안전! 충돌 위험한 보행자가 없습니다. (위치 좌표 :", veh_pt[i][0], ",", veh_pt[i][1], ")")
else:
    print("위험! 충돌 위험한 보행자가 있습니다! (위치 좌표 :", veh_pt[i][0], ",", veh_pt[i][1], ")")

변환행렬 h_ :
 [[-1.02324180e+01 -9.66125512e+01  3.71088613e+04]
 [ 2.18318626e+00 -2.49487913e+01 -1.06682612e+04]
 [-3.65919118e-03 -2.31359313e-02  1.00000000e+00]]
-10.232418017236835
2000


TypeError: can't multiply sequence by non-int of type 'numpy.float64'